In [7]:
import random
import numpy as np

In [140]:
def get_tree_ind(tiles, n=1):
    row = np.random.randint(low=0, high=tiles.shape[0], size=(n,))
    col = np.random.randint(low=0, high=tiles.shape[1], size=(n,))
    return zip(row, col)

def init_board(n_tile_rows, n_tile_cols, n_trees_init):
    tiles = np.zeros([n_tile_rows,n_tile_cols])
    trees = get_tree_ind(tiles, n_trees_init)
    for (r,c) in trees:
        tiles[r,c] += 1
    return tiles

def print_deck_info(deck):
    ncs = len([1 for x in deck if x == 'curse'])
    ngs = len(deck) - ncs
    print "Distribution: {0} curses, {1} gardens".format(ncs, ngs)

def init_deck(n_curses, n_gardens):
    gardens = ["garden" for i in xrange(n_gardens)]
    curses = ["curse" for i in xrange(n_curses)]
    deck = gardens + curses
    np.random.shuffle(deck)
    return deck

def shuffle_deck(deck, discard):
    assert len(deck) == 0
    deck = [x for x in discard]
    discard = []
    np.random.shuffle(deck)
    return deck, discard

def draw_card(deck, discard, is_verbose):
    if len(deck) == 0:
        if is_verbose:
            print "SHUFFLING."
        deck, discard = shuffle_deck(deck, discard)
    draw = deck.pop()
    discard.append(draw)
    return draw, deck, discard

def draw_garden(tiles, discard, is_verbose):
    ind = get_tree_ind(tiles, n=1)
    r,c = ind[0]
    tiles[r,c] += 1
    if is_verbose:
        print "Garden ({0},{1})".format(r,c)
    return tiles, discard

def valid_curse_inds(tiles):
    return [(r,c,tiles[r,c]) for r in xrange(tiles.shape[0]) for c in xrange(tiles.shape[1]) if tiles[r,c] > 0]
    
def choose_tree_ind(tiles):
    options = valid_curse_inds(tiles)
    top_opts = [(r,c) for r,c,nts in options if nts > 1]
    if top_opts:
        r,c = random.choice(top_opts)
        n = tiles[r,c]-1
    else:
        r,c = random.choice([(r,c) for r,c,nts in options])
        n = 1
    return r,c,n

def draw_curse(tiles, discard, is_verbose):
    r,c,n = choose_tree_ind(tiles)
    assert tiles[r,c] >= n
    tiles[r,c] -= n
    if tiles[r,c] == 0:
        discard.append("curse")
    if is_verbose:
        print "Cursing ({0},{1})".format(r,c)
    return tiles, discard


In [145]:
n_tile_rows = 6
n_tile_cols = 6
n_trees_init = 10
n_gardens = 5
n_curses = 3

n_games = 200
nplays = []
max_nplays = 500
is_verbose = False

for i in xrange(n_games):
    if is_verbose:
        print '================'
    
    deck = init_deck(n_curses, n_gardens)
    discard = []
    tiles = init_board(n_tile_rows, n_tile_cols, n_trees_init)

    c = 0
    while tiles.sum() > 0 and c < max_nplays:
        if len(deck) == 0 and is_verbose:
            print_deck_info(discard)
        if is_verbose:
            print tiles

        draw, deck, discard = draw_card(deck, discard, is_verbose)
        c += 1
        if draw == 'curse':
            tiles, discard = draw_curse(tiles, discard, is_verbose)
        elif draw == 'garden':
            tiles, discard = draw_garden(tiles, discard, is_verbose)

        if is_verbose:
            print '--------------'    
    if is_verbose:
        print "Game ended after {0} plays".format(c)
    nplays.append(c)
    if is_verbose:
        print '================'
        
print "Play counts: {0}".format(np.percentile(nplays, [25, 50, 75]))
print "{0} games did not complete".format(len([x for x in nplays if x == max_nplays]))


Play counts: [ 48.  52.  60.]
0 games did not complete


In [78]:
x = []
